In [14]:
from pyspark import SparkContext
from pyspark.mllib.recommendation import ALS
import math
import sys

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql.functions import isnan, isnull
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import functions as f

from pyspark.mllib.recommendation import ALS as rdd_als
from pyspark.sql.functions import lit
import math


In [28]:
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02
sqlContext=SQLContext(sc)

# read rating file
filename='s3://trendsmarketplacemsba2018fall/Ratings/ratings.csv'
colum1=[
    StructField("userId",LongType()),
    StructField("movieId",LongType()),
    StructField("rating",DoubleType()),
    StructField("timestamp",LongType())
]
schm1=StructType(colum1)
ratingDF=sqlContext.read.format('csv').schema(schm1).option("header",'true').load(filename)
ratingDF=ratingDF.select(ratingDF.userId, ratingDF.movieId, ratingDF.rating)
ratingDF.cache()

# read movie file
filename='s3://trendsmarketplacemsba2018fall/Movies/movies.csv'
colum=[
    StructField("movieId",LongType()),
    StructField("movieName",StringType()),
    StructField("genre",StringType())
]
schm=StructType(colum)
movieDF=sqlContext.read.format('csv').schema(schm).option("header",'true').load(filename)

# caculate movie review
movie_reviews_count=ratingDF.groupBy(ratingDF.movieId).agg({"movieId":"count", "rating":"avg"})
movie_reviews_count = movie_reviews_count.select("movieId", 
 f.col("avg(rating)").alias("Avg_Rating"),
 f.col("count(movieId)").alias("No_Reviews"))

#New user
# new_Id = 0
# new_user_ratings = [
 #     (0,260,4), # Star Wars (1977)
 #     (0,1,3), # Toy Story (1995)
 #     (0,16,3), # Casino (1995)
 #     (0,25,4), # Leaving Las Vegas (1995)
 #     (0,32,4), # Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
 #     (0,335,1), # Flintstones, The (1994)
 #     (0,379,1), # Timecop (1994)
 #     (0,296,3), # Pulp Fiction (1994)
 #     (0,858,5) , # Godfather, The (1972)
 #     (0,50,4) # Usual Suspects, The (1995)
 #    ]
 #new_user_rating_df=sqlContext.createDataFrame(new_user_ratings)


In [ ]:
inputname="s3://trendsmarketplacemsba2018fall/Input/new_user_0.csv"
outputname="s3://trendsmarketplacemsba2018fall/output/output_10"
get_recommendation(inputname,outputname,ratingDF,movieDF,movie_reviews_count)

In [75]:
def get_recommendation(inputname,outputname,ratingDF,movieDF,movie_reviews_count):    
    colum2=[
        StructField("userId",LongType()),
        StructField("movieId",LongType()),
        StructField("rating",DoubleType())]
    schm2=StructType(colum2)
    new_user_rating_df=sqlContext.read.format('csv').schema(schm2).option("header",'true').load(inputname)
    new_Id = [int(i.userId) for i in new_user_rating_df.select("userId").limit(1).collect()][0]
    
    #New rating form
    ratingDF_new = ratingDF.sample(False,0.99).union(new_user_rating_df)
    rating_rdd = ratingDF_new.rdd
    new_ratings_model = rdd_als.train(rating_rdd,5,10)

    ids = new_user_rating_df.select(new_user_rating_df.movieId)
    new_ids = [int(i.movieId) for i in ids.collect()]
    new_user_df =  movieDF.filter(movieDF.movieId.isin(*new_ids) == False)
    new_user_rdd=new_user_df.rdd.map(lambda x: (new_Id,x[0] ))

    new_user_recommendations_RDD = new_ratings_model.predictAll(new_user_rdd)
    new_user_recommendations_df=new_user_recommendations_RDD.toDF()
    final = new_user_recommendations_df.join(movie_reviews_count,
     new_user_recommendations_df.product == movie_reviews_count.movieId, 
     how="left")
    final = final.filter(final.No_Reviews >= 5000)
    final = final.sort("rating", ascending=False)
    final = final.select('movieId','rating','No_Reviews','Avg_Rating').join(movieDF, "movieId", how='left').limit(10)
    final.write.csv(outputname)#s3://trendsmarketplacemsba2018fall/output/output.csv
    return final